In [15]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

In [26]:
# Load the model

import pickle

path = r"models\xgb_model.pkl"
model = pickle.load(open(path, "rb"))




In [27]:
model.predict([[-0.65432992,  1.24504573,  0.23697162, -0.5001855 ,  0.56154562,
       -0.28664608, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
        0.94701966,  0.14260346,  0.30459857, -0.57427456,  1.71961809,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
        2.49494334, -0.41367015]])

array([1227.0487], dtype=float32)

In [28]:
data = [
      [ 1.07807977,  0.22192612,  0.77468297,  1.82005277, -1.26200749,
       -0.25194272, -0.12916836, -0.17535463, -0.22754885, -5.47803337,
       -0.64744078, -0.43640293,  0.30459857,  1.74132735, -0.58152447,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 ,  2.46842407,
       -0.40081071, -0.41367015],

      [-0.65432992,  1.24504573,  0.23697162, -0.5001855 ,  0.56154562,
       -0.28664608, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
        0.94701966,  0.14260346,  0.30459857, -0.57427456,  1.71961809,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
        2.49494334, -0.41367015],
    [ 0.50060987,  0.98718632, -0.69180254,  2.40011233,  0.92262892,
        1.56419996, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
       -0.98911087,  0.72160984,  0.30459857, -0.57427456, -0.58152447,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
       -0.40081071, -0.41367015]
    
]

prediction = model.predict(data)

In [29]:
prediction

array([ 298.6388, 1227.0487, 1316.8262], dtype=float32)

## Load SC

In [30]:
sc_dump_path = r"models\scaler.pkl"
sc = pickle.load(open(sc_dump_path, "rb"))

In [31]:
sc

StandardScaler()

## USER INPUT

In [32]:
Date = "11/06/2023"
Hour = 5
Temperature = 34
Humidity = 55
Wind_speed = 67
visibility = 100
Solar_Radiation = 0.4
Rainfall = 0.7
Snowfall = 0.2
seasons  = 'Summer'
Holiday = "No Holiday"
Functioning_Day = "Yes"



## Converting User Data into Model Consumable Formate

In [33]:
from datetime import datetime

def get_string_to_datetime(Date):
    dt = datetime.strptime(Date, "%d/%m/%Y")
    return {"day": dt.day, "month": dt.month, "year": dt.year, "weekday": dt.strftime("%A")}

# Now you can call the function

str_to_date = get_string_to_datetime(Date)
str_to_date 

{'day': 11, 'month': 6, 'year': 2023, 'weekday': 'Sunday'}

In [34]:
holiday_dict = {"No Holiday":0, "Holiday":1}
functioning_day_dict = {"No":0, "Yes":1}

In [35]:
u_input_list = [Hour,Temperature,Humidity,Wind_speed,visibility,Solar_Radiation,Rainfall,Snowfall ,
                holiday_dict[Holiday],functioning_day_dict[Functioning_Day], str_to_date["day"],str_to_date["month"],str_to_date["year"],]
features_name = ['Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day',
       'Month', 'Year'
       ]
df_u_input = pd.DataFrame([u_input_list], columns=features_name)
df_u_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,5,34,55,67,100,0.4,0.7,0.2,0,1,11,6,2023


In [36]:
u_input_list

[5, 34, 55, 67, 100, 0.4, 0.7, 0.2, 0, 1, 11, 6, 2023]

In [37]:
def season_to_df(season):
    
    seasons_col = ['Spring', 'Summer', 'Winter']
    seasons_data = np.zeros((1,len(seasons_col)), dtype ="int")
    df_seasons = pd.DataFrame(seasons_data, columns = seasons_col)

    if seasons in seasons_col:
        df_seasons[seasons]=1
    return df_seasons 

seasons_df = season_to_df(seasons)
seasons_df

,Spring,Summer,Winter
0,0,1,0


In [38]:
def days_name(weekday):
    days_names = ['Monday', 'Saturday','Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    days_name_data = np.zeros((1,len(days_names)), dtype="int")
    df_days = pd.DataFrame(days_name_data, columns=days_names)

    if weekday in days_names:
        df_days[weekday] = 1
    return df_days

days_name_df = days_name(str_to_date["weekday"])
days_name_df                                          

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,1,0,0,0


In [39]:
df_final_input = pd.concat([df_u_input,seasons_df,days_name_df ], axis=1)
df_final_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,5,34,55,67,100,0.4,0.7,0.2,0,1,...,2023,0,1,0,0,0,1,0,0,0


In [40]:
sc_data_for_pred = sc.transform(df_final_input)
sc_data_for_pred

array([[-0.94306487,  1.76076456, -0.15409119, 63.11301355, -2.21005725,
        -0.19410378,  0.46358031,  0.29083855,  0.        ,  0.18254726,
        -0.53355074, -0.14689974, 18.24263884, -0.57427456,  1.71961809,
        -0.57866842, -0.40630934, -0.40916536,  2.45398535, -0.40511677,
        -0.40081071, -0.41367015]])

## Prediction in Deployment

In [41]:

print(f"Rented Bike Demand on date: {Date}, and Time: {Hour} is: {round(model.predict(sc_data_for_pred).tolist()[0])}")



Rented Bike Demand on date: 11/06/2023, and Time: 5 is: 108
